<b> Part 1: Scrape articles on tolo news from May to August<b>

In [1]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

In [2]:
#determine start range and end rage of articles (1871830 is around mid may and 171830 august after the taliban took over kabul
start = 171830
end = 174273

articles = pd.DataFrame()
while start < end:
    try:
        url = 'https://tolonews.com/afghanistan-'+str(start)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html.parser")
        title = soup.find("h2",attrs={'class':'grey-dark1 title-top-post-tolonews'}).text
        body = soup.find("div", attrs={'class':'views-field views-field-field-article-body-summary-1 full-html-text'}).text
        date = json.loads(soup.find("script",type="application/ld+json").text)
        date = date['@graph'][0]['datePublished']
        combine = title,body,date,url
        df = pd.DataFrame(combine).T.rename(columns={0:"Title",1:"Body",2:"Date",3:"URL"})
        articles = articles.append(df)
    except:
articles.reset_index(keep=False,inplace=True)

https://tolonews.com/afghanistan-171830 200
https://tolonews.com/afghanistan-171831 404
https://tolonews.com/afghanistan-171832 404
https://tolonews.com/afghanistan-171833 404
https://tolonews.com/afghanistan-171834 200
https://tolonews.com/afghanistan-171835 200
https://tolonews.com/afghanistan-171836 404
https://tolonews.com/afghanistan-171837 404
https://tolonews.com/afghanistan-171838 200
https://tolonews.com/afghanistan-171839 404
https://tolonews.com/afghanistan-171840 404
https://tolonews.com/afghanistan-171841 404
https://tolonews.com/afghanistan-171842 200
https://tolonews.com/afghanistan-171843 404
https://tolonews.com/afghanistan-171844 404
https://tolonews.com/afghanistan-171845 200
https://tolonews.com/afghanistan-171846 404
https://tolonews.com/afghanistan-171847 404
https://tolonews.com/afghanistan-171848 404
https://tolonews.com/afghanistan-171849 200
https://tolonews.com/afghanistan-171850 200
https://tolonews.com/afghanistan-171851 404
https://tolonews.com/afghanistan

<b> Part 2: filter articles with taliban take over information <b>

In [2]:
import pandas as pd 
import string
import re
import numpy as np
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from fuzzywuzzy import process

/Users/anniestishov/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [15]:
#load files
articles = pd.read_csv(r'/Users/anniestishov/Desktop/tadhg_python_projects/Afghanistan District Take over/articles.csv',index_col='Unnamed: 0')
#key_words
key_words = ['seize','march','fall','take over','advance','capture','attack','surround','move into','threat'
             ,'control','taliban','capital','cities','district','province','control']

In [264]:
# Function to preprocess and clean text
def clean_text(x):
    cleaned_text = list()
    for i in x:
        i = i.rstrip('\n')
        i = i.replace('| TOLOnews','')
        i = ''.join([s for s in i if not s.isdigit()])
        i = i.translate(str.maketrans("","", string.punctuation))
        i = re.sub('[^a-zA-Z0-9 \n\.]', '',i)
        i = i.lower()
        cleaned_text.append(i)
    return cleaned_text

In [265]:
#get article title and clean 
article_title = list(articles['Title'])
article_title_cleaned = clean_text(article_title)

In [256]:
#function to tokenize and remove stopwords
def tokenize_remove_stopword(text):
    # tokenize text
    tokenized_text = list()
    for entry in text:
        token = word_tokenize(entry)
        tokenized_text.append(token)
    #remove stopwords
    stop_words = set(stopwords.words('english'))
    tokenized_text_stopword = []
    for entry in tokenized_text:
        temp = []
        for word in entry:
            if word not in stop_words:
                temp.append(word)
        tokenized_text_stopword.append(temp)
    return tokenized_text_stopword

In [259]:
#apply tokenize and wordstop removal function to cleaned article title
article_title_preprocessed = tokenize_remove_stopword(article_title_cleaned)

In [260]:
# function to compare text to list of keywords and return the closest match:
def text_similarity_func(text,keywords,score_cutoff):
    text_keyword_match = list()
    for j in text:
        temp = list()
        for i in j:
            try:
                temp.append(process.extractOne(i, keywords,score_cutoff = score_cutoff)[0])
            except:
                pass
        if not temp:
            temp = np.nan
        text_keyword_match.append(temp)
    return text_keyword_match

In [261]:
#apply similarity function to cleaned and tokenized title text
title_keyword_match = text_similarity_func(article_title_preprocessed,key_words,80)

In [262]:
# convert to dataframe and map to articles_df
articles['Title_KeyWord_match'] = title_keyword_match
# Drop Nas
articles_reduced = articles.dropna().reset_index(drop=True)

In [247]:
#load Taliban districts and clean 
districts = pd.read_csv(r'/Users/anniestishov/Desktop/tadhg_python_projects/Afghanistan District Take over/afghanistan_districts.txt',sep='\t')
districts_list = list(districts['District'])
districts_list_clean = clean_text(districts_list)

In [272]:
#clean article text of articles_reduced df
article_reduced_body = list(articles_reduced['Text'])
article_reduced_body_clean = clean_text(article_reduced_body)

In [368]:
#tokenize, remove stop words and keep noumns
article_body_tokenized = tokenize_remove_stopword(article_reduced_body_clean)
#POS tag keep tokens start with N,idea is to shorten list of words to compare to since we are only trying to
# yield the district which are nouns
articles_nouns = list()
for i in article_body_tokenized:
    nouns = list()
    for pos in pos_tag(i):
        if pos[1].startswith('N'):
            nouns.append(pos[0])
    if not nouns:
            nouns = np.nan
    articles_nouns.append(nouns)
# compare to district list and return close districts
afghan_district_match = text_similarity_func(articles_nouns,districts_list_clean,95)   

In [376]:
# convert to dataframe and map to articles_reduced_df
articles_reduced['Text_district_match'] = afghan_district_match
# export to csv 
articles_reduced.to_csv(r'/Users/anniestishov/Desktop/tadhg_python_projects/Afghanistan District Take over/articles_reduced_v2_95.csv')

<b> Part 3: Create a time series map of the taliban taking over districts in Afghanistan

After exporting article_reduced_v2_95 I did a bit of manual work: I reviewed the 210 articles that the processed tagged and manually reviewed them. I compared the districts in the text to the one identified in the "fallen district" field and added or removed those that it missed. I then identified which articles reported the same district and removed the duplicate. I ended up with a short list of 198 districts that fell to the taliban from May to August

using a publicly available geojson file of afghanistan's districts I mapped as many of those districts as possible.

I did a bit more cleaning work in excel and ended up with this table bellow

In [4]:
data_to_map = pd.read_excel(r'/Users/anniestishov/Desktop/tadhg_python_projects/Afghanistan District Take over/outputs/district398_taliban_control_dbf.xlsm',sheet_name="data_for_map")

In [9]:
data_to_map

,District,OBJECTID,PROV_34_NA,DISTID,PROVID,2021-05-11 00:00:00,2021-05-12 00:00:00,2021-06-07 00:00:00,2021-06-08 00:00:00,2021-06-12 00:00:00,...,2021-07-25 00:00:00,2021-08-06 00:00:00,2021-08-07 00:00:00,2021-08-08 00:00:00,2021-08-10 00:00:00,2021-08-11 00:00:00,2021-08-12 00:00:00,2021-08-13 00:00:00,2021-08-14 00:00:00,2021-08-16 00:00:00
0,Ab Band,209,Ghazni,611,6,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,Ab Kamari,54,Badghis,1902,19,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
2,Acheen,132,Nangarhar,810,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Adraskan,112,Hirat,2011,20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ahmad Abad,147,Paktya,705,7,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,Zari,321,Balkh,1603,16,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
387,Zebak,246,Badakhshan,1105,11,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
388,Zhari,52,Kandahar,2416,24,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
389,Ziruk,187,Paktika,2912,29,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


The idea is to have every day reported by tolo news as column and return a 1 for when a district was reported to have fallen to the taliban and keep it as 1 throughout the columns. Looping through the columns we will see the gradual takeover of afghanistan ending in kabul. I used tableau to vizual the districts, for it I downloaded a geo.json file of the ditricts, joined this file to it and looped through the columns to show the time progression. See tableau file for more details

Tableau dashboard: Taliban takeover of Afghanistan reported by Tolo News:
tableau link
https://public.tableau.com/views/TalibanTakeOverofAfghanistan-CaseStudyusingToloNewsReorting/TalibanTakeOver?:language=en-US&:display_count=n&:origin=viz_share_link